In [1]:
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("../azureml-models/bi_cross_encoders/1/models/bi_encoder")

ds = load_from_disk("../data/processed/corpus_all_length_250_multi-qa-mpnet-base-dot-v1_TRAINED_2022-10-21-14-33-13_version_45/ds_with_embeddings.hf/")

In [2]:
tokenizer = model.tokenizer
print(ds)

Dataset({
    features: ['_id', 'doc_id', 'title', 'paragraph_id', 'text', 'chunked', 'embeddings', 'chunk_id', 'idx'],
    num_rows: 3703456
})


## Code used to extend the dataset with extra columns needed to expand the context

In [ ]:
from difflib import SequenceMatcher


def map_fn(sample, idx):
    """
    Add chunk_id column, that is the index of the paragraph before it that belongs together, the first one will take the index of itself
    If there is no chunk, the chunk_id is set to -1

    Args:
        sample (Dict): Row of the dataset
        idx (int): index of the row

    Returns:
        Dict: new updated row
    """
    # if first element is chunked, set chunk_id (can't check previous element)
    if idx == 0 and sample["chunked"]:
        # print("First element is chunked")
        sample["chunk_id"] = idx
        return sample
    
    # If not the first element, but still chunked 
    if sample["chunked"]:
        # print(f"sample {idx} is chunked")
        chunk_id = idx
        for i in range(idx-1, -1, -1):
            # print(f"    checking if {i} is chunked")
            # check if previous element is chunked
            # if yes, check if the overlap is big enough
            # and change the chunk_id to the index of that previous element
            if ds[i]["chunked"]:
                previous_text = ds[i]["text"][-150:]
                current_text = ds[i+1]["text"][:150]
                
                sm = SequenceMatcher(None, previous_text, current_text)
                a,b,s = sm.find_longest_match(0, len(previous_text), 0, len(current_text))
                
                if s > 19 and b==0 and a==len(previous_text)-s:
                    # print(f"        sample {i} is overlapping with {idx}")
                    chunk_id = i
                    continue
                else: # if there is not enough overlap, we know that it isn't part of the same chunked paragraph, but another one so we don't take its index
                    # print("        not enough overlap")
                    break    
            # if previous element is not chunked, we know that it isn't part of the same chunked paragraph thus we stop the loop and don't change the chunk_id
            else:
                # print("        previous element is not chunked, stop loop")
                break
        
        sample["chunk_id"] = chunk_id
        return sample
    else:
        sample["chunk_id"] = -1
        return sample

ds_chunk_ids = ds.map(map_fn, with_indices=True)     

def add_idx(sample, idx):
    sample["idx"] = idx
    return sample

ds = ds.map(add_idx, with_indices=True)

In [3]:
ds.add_faiss_index(column='embeddings')

  0%|          | 0/3704 [00:00<?, ?it/s]

Dataset({
    features: ['_id', 'doc_id', 'title', 'paragraph_id', 'text', 'chunked', 'embeddings', 'chunk_id', 'idx'],
    num_rows: 3703456
})

In [4]:
query = ["Tensions between Turkey and Greece?"]

query_embedding = model.encode(query, convert_to_tensor=False, convert_to_numpy=True)
_scores, samples = ds.get_nearest_examples('embeddings', query_embedding, k=128)

In [22]:
from typing import List

class Context():
    def __init__(self, paragraph_idx:int, idxs_before:List[int], idxs_after:List[int], text:str):
        self.text = text
        self.paragraph_idx = paragraph_idx
        self.idxs_before = idxs_before
        self.idxs_after = idxs_after
        self.duplicate = False
        
        
    def __eq__(self, other):
        if isinstance(other, Context):
            all_idxs_self = self.idxs_before + [self.paragraph_idx] + self.idxs_after
            all_idxs_other = other.idxs_before + [other.paragraph_idx] + other.idxs_after
            
            # sort just to be sure as [1,2,3] != [1,3,2] but should be equal in our case
            all_idxs_self.sort()
            all_idxs_other.sort()
            return all_idxs_self == all_idxs_other
        return False
    
    def __str__(self) -> str:
        return self.text if not self.duplicate else "DUPLICATE"

In [39]:
from difflib import SequenceMatcher
from typing import Dict

def __remove_text_overlap(texts:List[str], idx:int):
    t = [texts[0]]
    for i in range(1, len(texts)):
        # last 150 characters of previous text and first 150 characters of current text
        previous_text = t[i-1][-150:]
        current_text = texts[i][:150]
        
        sm = SequenceMatcher(None, previous_text, current_text)
        a,b,s = sm.find_longest_match(0, len(previous_text), 0, len(current_text))
        
        # overlapping length is long enough and the overlap is at the beginning of the current text and the end of the previous text
        if s > 19 and b==0 and a==len(previous_text)-s:
            removed_overlap = texts[i][s:]
            t.append(removed_overlap)
        else:
            raise Exception("Overlap is not long enough or not at the beginning of the current text")
    return t

def __connect_overlapping_chunks(context_dict:Dict, idx:int):
    texts = context_dict["texts"]
    t = __remove_text_overlap(texts, idx)
    
    # truncate chunks to less than 1800 tokens with the idx in the middle
    text_before = " ".join(t[:idx])
    text_paragraph = t[idx]
    text_after = " ".join(t[idx+1:])
    
    # keep track of the idxs of the different chunks
    idxs_before = context_dict["idxs"][:idx]
    idxs_paragraph = context_dict["idxs"][idx]
    idxs_after = context_dict["idxs"][idx+1:]
    
    # make sure we splitted the idxs correctly
    assert idxs_paragraph == context_dict["paragraph_idx"]
    
    # tokenize the different paragraphs (without overlap)
    tokens = tokenizer(t)["input_ids"]
    tokens_per_chunk_before = tokens[:idx]
    tokens_paragraph = tokens[idx]
    tokens_per_chunk_after = tokens[idx+1:]
    
    # get the total amount of tokens before and after the paragraph that was returned by semantic search
    amount_tokens_before = sum((len(tokens) for tokens in tokens_per_chunk_before))
    amount_tokens_after = sum((len(tokens) for tokens in tokens_per_chunk_after))
    
    # if the total size of the context is less than 1800 tokens, we can keep it all
    if  amount_tokens_before + len(tokens_paragraph) + amount_tokens_after < 1800:
        full_text = " ".join([text_before, text_paragraph, text_after])
        c = Context(paragraph_idx=idxs_paragraph, idxs_before=idxs_before, idxs_after=idxs_after, text=full_text)
        
        return c
    else:
        # truncate the context before and after the paragraph (the paragraph is always kept as is)
        amount_tokens_context_tokens = 1800 - len(tokens_paragraph)
        # ideal amount of tokens per side
        balanced_amount_tokens_per_side = amount_tokens_context_tokens // 2
        
        
        # Incrementally add tokens to the context until we reach the desired amount of tokens
        # This way we can keep track of which paragraphs we added to the context and which ones are completely discarded
        total_tokens = 0
        accepted_tokens_before = []
        accepted_tokens_after = []        
        accepted_idxs_before = []
        accepted_idxs_after = []
        
        # Add chunks to the front and back simulataneously until we reach the desired amount of tokens
        while total_tokens < amount_tokens_context_tokens:
            if len(tokens_per_chunk_before) > 0:
                accepted_tokens_before = tokens_per_chunk_before.pop() + accepted_tokens_before
                accepted_idxs_before = [idxs_before.pop()] + accepted_idxs_before
            
            if len(tokens_per_chunk_after) > 0:
                accepted_tokens_after += tokens_per_chunk_after.pop(0)
                accepted_idxs_after.append(idxs_after.pop(0))
            total_tokens = len(accepted_tokens_before) + len(accepted_tokens_after)
        
        # paragraphs are big so we might have added too many tokens, so we need to remove some of the last paragraph in the front and back
        reduced_tokens_before = accepted_tokens_before[-balanced_amount_tokens_per_side:]
        reduced_tokens_after = accepted_tokens_after[:balanced_amount_tokens_per_side]
        
        # One of the sides might be longer than the other (for example when the paragraph was located at the bottom of the complete context), 
        # so we need to add tokens to the other side to make sure we completely fill the context
        extra_needed_before = balanced_amount_tokens_per_side - len(reduced_tokens_after)
        extra_needed_after = balanced_amount_tokens_per_side - len(reduced_tokens_before)
        
        tokens_before = accepted_tokens_before[-balanced_amount_tokens_per_side-extra_needed_before:]
        tokens_after = accepted_tokens_after[:balanced_amount_tokens_per_side+extra_needed_after]
        
        # decode the tokens back to text 
        # (the first and last paragraph might be truncated so we need to decode so that we know which text exactly is in the context)
        decoded_text_before = tokenizer.decode(tokens_before, skip_special_tokens=True)
        decoded_text_after = tokenizer.decode(tokens_after, skip_special_tokens=True)
        
        full_text = " ".join([decoded_text_before, t[idx], decoded_text_after])
        
        c = Context(paragraph_idx=idxs_paragraph, idxs_before=accepted_idxs_before, idxs_after=accepted_idxs_after, text=full_text)
        
        return c

def expand_context(ds, og_samples):
    """
    Function that expands paragraphs to a context. It will look for all paragraphs that have the same chunk_id and pass all these paragraphs to the
    __connect_overlapping_chunks function.

    Args:
        ds (Dataset): Huggingface Dataset that has all the paragraphs. It should have a column called "chunk_id" that indicates which chunk a paragraph belongs to. 
        og_samples (Dict): Result returned by the semantic search
        
    Returns:
        List: List of contexts
    """
    samples = og_samples.copy()
    context = []
    for i in range(len(samples["idx"])):
        if samples["chunk_id"][i] != -1:
            chunk_id = samples["chunk_id"][i]
            
            look_ahead_idx = samples["idx"][i] + 1
            while ds[look_ahead_idx]["chunk_id"] == chunk_id:
                look_ahead_idx += 1
            
            context_dict = {
                "texts": [ds[j]['text'] for j in range(chunk_id, look_ahead_idx)],
                "idxs": [ds[j]['idx'] for j in range(chunk_id, look_ahead_idx)],
                "paragraph_idx": samples["idx"][i],
            }
            
            assert context_dict["paragraph_idx"] == context_dict["idxs"][samples["idx"][i]- chunk_id]
            
            ctx = __connect_overlapping_chunks(context_dict, samples["idx"][i]- chunk_id)
            
            context.append(ctx)
        
        else:
            c = Context(paragraph_idx=samples["idx"][i], idxs_before=None, idxs_after=None, text=samples["text"][i])
            context.append(c)
        
    
    samples["context"] = context
    return samples

In [40]:
expanded_samples = expand_context(ds, samples)

In [41]:
expanded_samples["context"][0].text

'We welcome the steady improvement in the bilateral relations between Greece and Turkey. EU is looking forward to further progress in this area. Must be based on the principle of peaceful settlement of disputes, in accordance with the Helsinki conclusions and the Accession Partnership. Welcome the recent exploratory contacts between Greece and Turkey in this field.'

In [15]:
# find amount of token in each sample context
for i in range(len(expanded_samples["context"])):
    if len(tokenizer(expanded_samples["context"][i].text)["input_ids"]) >1800:
        print(i)
        print(len(tokenizer(expanded_samples["context"][i].text)["input_ids"]))

## Test

In [42]:
# data = zip(expanded_samples["chunk_id"], expanded_samples["context"])


chunked_sample_idxs = [i for i, c_id in enumerate(expanded_samples["chunk_id"]) if c_id != -1]
print("Chunked samples: ", chunked_sample_idxs)
# first one can't be a duplicate
for num, i in enumerate(chunked_sample_idxs[1:], 1):
    chunk_id = expanded_samples["chunk_id"][i]
    # print(f"all previous samples to check: {list(chunked_sample_idxs[:num])}")
    # compare to all previous samples
    for j in chunked_sample_idxs[:num]:
        ci = expanded_samples["context"][i]
        cj = expanded_samples["context"][j]
        # print(f"Compare {i} with {j}")
        
        if ci == cj:
            print(f"{i} is a duplicate of {j}")
            ci.duplicate = True
            break
    # print("_"*100)
                

Chunked samples:  [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 66, 67, 68, 69, 70, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]
8 is a duplicate of 2
17 is a duplicate of 1
19 is a duplicate of 3
23 is a duplicate of 5
29 is a duplicate of 28
30 is a duplicate of 4
31 is a duplicate of 15
42 is a duplicate of 2
53 is a duplicate of 49
61 is a duplicate of 2
77 is a duplicate of 70
80 is a duplicate of 5
86 is a duplicate of 27
92 is a duplicate of 69
95 is a duplicate of 64
99 is a duplicate of 2
118 is a duplicate of 38
120 is a duplicate of 106


In [38]:
str(expanded_samples["context"][19])

'DUPLICATE'

In [9]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [11]:
s1 = 'COMMISSION STAFF WORKING DOCUMENT Turkey 2020 Report Accompanying the Communication from the Commission to the European Parliament, the Council, the European Economic and Social Committee and the Committee of the Regions 2020 Communication on EU Enlargement Policy to play an active role in supporting the negotiations, including by appointing, upon resumption, a representative to the UN Good Offices Mission. The process of granting the Committee on Missing Persons full access to all relevant archives and military areas has seen welcome developments, which need to be followed up by concrete actions and further expedited. Despite repeated calls by the Council and the Commission, Turkey has still not complied with its obligations as outlined in the Declaration of the European Community and its Member States of 21 September 2005 and in Council Conclusions, including those of December 2006 and December 2015. Turkey has not fulfilled its obligation to ensure full and non-discriminatory implementation of the Additional Protocol to the Association Agreement and has not removed all obstacles to the free movement of goods, including restrictions on direct transport links with the Republic of Cyprus. Turkey continued to veto applications by the Republic of Cyprus to join several international organisations, including the Organisation for Economic Co-operation and Development (OECD). In January 2020, Turkey blocked the Republic of Cyprus\' participation as an observer at the United Nations Conference on Disarmament. Peaceful settlement of border disputes In September 2020, Greece and Turkey agreed to re-launch bilateral exploratory talks, which should remain a valuable channel of communication between both sides to find common ground for the start of negotiations on the delimitation of the continental shelf. The EU welcomed the confidence building steps by Greece and Turkey, as well as the announcement that they will resume their direct exploratory talks aiming at the delimitation of the Continental Shelf and Exclusive Economic Zone of the two countries.These efforts should be sustained and broadened. The signing of a bilateral Memorandum of Understanding on the delimitation of maritime jurisdiction areas between Turkey and the National Accord Government of Libya in November 2019 increased tensions in the Eastern Mediterranean, as it ignored the sovereign rights of 2019, unequivocally reaffirmed its solidarity with Greece and Cyprus regarding actions by Turkey in the Eastern Mediterranean and the Aegean Sea. It stressed that the Memorandum infringes upon the sovereign rights of third States, does not comply with the United Nations Convention on the Law of the Sea and cannot produce any legal consequences for third States. Furthermore, the EU stated that the sovereignty and sovereign rights over the maritime zones of all neighbouring coastal states, including those generated by their islands, need to be respected. The delimitation of exclusive economic zones and continental shelf should be addressed through Cyprus and Greece, underlining that abstaining from unilateral actions is a basic element to serious negative impact across the range of EU-Turkey relations. The launch of a tender for offshore oil and gas exploration, including part of the continental shelf off the Greek island of Castellorizo, continued to be objected by Greece. Aiming at emphasising its territorial claims, Turkey unilaterally deployed in August and up until 12 September 2020 a survey and military ships in the continental shelf off Castellorizo. In May 2020, the Turkish Petroleum Corporation submitted applications to the Turkish Government for the granting of exploration and exploitation permits south and south-east of four Greek islands. The threat of casus belli in relation to the possible extension of Greek territorial sea to 12 nautical miles, as set out in a 1995 Turkish Grand National Assembly resolution, still stands, and was reiterated in relation to the possible extension of the Greek territorial waters in the'
s2 = 'COMMISSION STAFF WORKING DOCUMENT Turkey 2018 Report Accompanying the document Communication from the Commission to the European Parliament, the Council, the European Economic and Social Committee and the Committee of the Regions 2018 Communication on EU Enlargement Policy resolutions and in line with the principles on which the EU is founded. It is important to preserve the progress made so far and to pursue preparations for a fair, comprehensive and viable settlement, including in its external aspects. Turkey’s commitment and contribution in concrete terms to this comprehensive settlement remains crucial. The process of granting the Committee on Missing Persons full access to all relevant archives and military areas has seen welcome developments but needs to be further expedited. Despite repeated calls by the Council and the Commission, Turkey has still not complied with its obligations as outlined in the Declaration of the European Community and its Member States of 21 September 2005 and in Council Conclusions, including those of December 2006 and December 2015. Turkey has not fulfilled its obligation to ensure full and non-discriminatory implementation of the Additional Protocol to the Association Agreement and has not removed all obstacles to the free movement of goods, including restrictions on direct transport links with the Republic of Cyprus. There was no progress on normalising bilateral relations with the Republic of Cyprus. Turkey continued to veto applications by the Republic of Cyprus to join several international organisations, including the Organisation for Economic Co-operation and Development (OECD). Peaceful settlement of border disputes Bilateral exploratory talks were held between Greece and Turkey to find common ground for the start of negotiations on the delimitation of the continental shelf. Greece continued to object to Turkey’s launch of a tender for offshore oil and gas exploration including part of the continental shelf off the Greek island of Castellorizo. The threat of casus belli in relation to the possible extension of Greek territorial waters, as made in a Turkish Grand National Assembly resolution in 1995, still stands. President Erdoğan visited Greece in December 2017, the first visit by a Turkish President in 65 years. However, statements made before and during his visit to Greece with regards to the Treaty of Lausanne caused public controversy. The agreement to ease increasing tensions in the Aegean Sea reached during the visit between the Turkish President and the Greek Prime Minister was not followed up in practice, as in February 2018, a series of incidents took place which culminated in the collision of a Hellenic Coast Guard vessel, co-financed by the European Border and Coast Guard, by a Turkish patrol boat off the islets of Imia. Another source of serious concern has been the detention of two Greek soldiers who were patrolling the land border between Greece and Turkey. Greece and Cyprus made formal complaints about repeated and increased violations of their territorial waters and airspace by Turkey, including flights over Greek islands. In line with the Negotiating Framework and previous European Council and Council conclusions, the Council has reiterated that Turkey needs to commit itself unequivocally to good neighbourly relations, international agreements and the peaceful settlement of disputes in accordance with the United Nations Charter, having recourse, if necessary, to the International Court of Justice. In this context, the EU has again expressed serious concern, and urged Turkey to avoid any kind of threat or action directed against a Member State, or any source of friction or action that would damage good neighbourly relations and the peaceful settlement of disputes. Regional cooperation Good neighbourly relations form an essential part of Turkey’s process of moving towards the EU. Bilateral relations with other enlargement countries and neighbouring EU Member States were generally good. Cooperation with Greece and Bulgaria on migration furthe'

similar(s1, s2)

0.2275301649839941